In [321]:
#importing all the necessary libraries required
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [322]:
df = pd.read_csv("C:/Users/pande/Desktop/kaggle/train.csv")

In [323]:
#so now after estimation now we will make a function to replace all missing values of age
def replace_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

In [324]:
#now applying our fucntion created on the Age column
df['Age'] = df[["Age","Pclass"]].apply(replace_age,axis=1)

In [325]:
df.drop(["Cabin"],axis=1,inplace=True)

In [326]:
df.dropna(inplace=True)

In [327]:
#now will be transforming categorical variables to continuous values
sex = pd.get_dummies(df['Sex'],drop_first=True)
embark = pd.get_dummies(df['Embarked'],drop_first=True)

In [328]:
df.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [329]:
# now adding continuous columns for all categorical variables
df = pd.concat([df,sex,embark],axis=1)

In [330]:
df.drop(["PassengerId"],axis=1,inplace=True)

In [331]:
#now spliting dataset in independent and dependent(target) variables

X = df.drop("Survived",axis=1)
y = df["Survived"]

In [332]:
y.value_counts()

0    549
1    340
Name: Survived, dtype: int64

In [333]:
#now will divide data on test and train modules
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=1)

In [334]:
from sklearn.ensemble import GradientBoostingClassifier
gbr = GradientBoostingClassifier()
gbr.fit(X_train,y_train)

GradientBoostingClassifier()

In [335]:
gbr.score(X_train,y_train)

0.8903903903903904

In [336]:
gbr.score(X_test,y_test)

0.8385650224215246

In [337]:
from sklearn.model_selection import RandomizedSearchCV
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
random_grid ={'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
              'n_estimators':[100,250,500,750,1000,1250,1500,1750],
              'max_depth':max_depth,
              'min_samples_split':[2,4,6,8,10,20,40,60,100], 
              'min_samples_leaf':[1,3,5,7,9],
              'max_features':['auto','sqrt'],
              'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1],
              'max_leaf_nodes': [2,5,10]}

In [338]:
rf_random = RandomizedSearchCV(estimator = gbr, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [339]:
rf_random.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.2min finished


RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.15, 0.1, 0.05, 0.01,
                                                          0.005, 0.001],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'max_leaf_nodes': [2, 5, 10],
                                        'min_samples_leaf': [1, 3, 5, 7, 9],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              20, 40, 60, 100],
                                        'n_estimators': [100, 250, 500, 750,
                                                         1000, 1250,

In [340]:
rf_random.best_params_

{'subsample': 0.7,
 'n_estimators': 1250,
 'min_samples_split': 8,
 'min_samples_leaf': 3,
 'max_leaf_nodes': 10,
 'max_features': 'sqrt',
 'max_depth': 60,
 'learning_rate': 0.001}

In [341]:
gbc = GradientBoostingClassifier(subsample=0.7,learning_rate=0.001,n_estimators=1250,min_samples_leaf=3,max_features='sqrt',max_depth=60,min_samples_split=8,max_leaf_nodes=10)

In [342]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.001, max_depth=60,
                           max_features='sqrt', max_leaf_nodes=10,
                           min_samples_leaf=3, min_samples_split=8,
                           n_estimators=1250, subsample=0.7)

In [343]:
gbc.score(X_train,y_train)

0.8438438438438438

In [344]:
gbc.score(X_test,y_test)

0.8385650224215246

In [345]:
y_pred = gbc.predict(X_test)

In [346]:
from sklearn.metrics import classification_report
s = classification_report(y_test,y_pred)

In [347]:
print(s)

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       138
           1       0.89      0.66      0.76        85

    accuracy                           0.84       223
   macro avg       0.85      0.80      0.82       223
weighted avg       0.85      0.84      0.83       223



In [348]:
from xgboost import XGBClassifier


In [349]:
xgc = XGBClassifier()
xgc.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [350]:
xgc.score(X_train,y_train)

0.9714714714714715

In [351]:
xgc.score(X_test,y_test)

0.8295964125560538

In [352]:
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
random_grid1 ={ 'max_depth':max_depth,
               'min_child_weight':range(1,6),
               'gamma':[i/10.0 for i in range(0,10)],
               'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1],
               'reg_alpha':[0,0.1, 1, 100],
               'reg_lambda':[0,0.1, 1, 100],
               'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001],
               'n_estimators':[100,250,500,750,1000,1250,1500,1750]
                }

In [353]:
rf_random1 = RandomizedSearchCV(estimator = xgc, param_distributions = random_grid1, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [354]:
rf_random1.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.3min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           gpu_id=-1, importance_type='gain',
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_delta_step=0, max_depth=6,
                                           min_child_weight=1, missing=nan,
                                           monotone_constraints='()',
                                           n_estimators=100, n_jobs=0,
                                           num_pa...
                   param_distributions={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                                  0.6, 0

In [355]:
rf_random1.best_params_

{'subsample': 0.75,
 'reg_lambda': 1,
 'reg_alpha': 1,
 'n_estimators': 100,
 'min_child_weight': 1,
 'max_depth': 20,
 'learning_rate': 0.15,
 'gamma': 0.8}

In [356]:
xgbc = XGBClassifier(subsample=0.4,reg_alpha=0.001,reg_lambda=10,n_estimators=1500,min_child_weight=4,max_depth=100,learning_rate=0.001,gamma=0.1)

In [357]:
xgbc.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.001, max_delta_step=0, max_depth=100,
              min_child_weight=4, missing=nan, monotone_constraints='()',
              n_estimators=1500, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0.001, reg_lambda=10, scale_pos_weight=1, subsample=0.4,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [358]:
xgbc.score(X_train,y_train)

0.8033033033033034

In [359]:
xgbc.score(X_test,y_test)

0.8475336322869955

In [360]:
y_predfff = xgbc.predict(X_test)

In [361]:
from sklearn.metrics import classification_report
s = classification_report(y_test,y_pred)

In [362]:
print(s)

              precision    recall  f1-score   support

           0       0.82      0.95      0.88       138
           1       0.89      0.66      0.76        85

    accuracy                           0.84       223
   macro avg       0.85      0.80      0.82       223
weighted avg       0.85      0.84      0.83       223



In [363]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=1000)
log.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [364]:
log.score(X_train,y_train)

0.7912912912912913

In [365]:
log.score(X_test,y_test)

0.852017937219731

In [366]:
from sklearn.preprocessing import StandardScaler
poly = StandardScaler()

In [367]:
poly.fit(X_train)

StandardScaler()

In [368]:
X_train_p = poly.transform(X_train)
X_test_p  = poly.transform(X_test)

In [369]:
log1 = LogisticRegression(max_iter=10000)
log1.fit(X_train_p,y_train)

LogisticRegression(max_iter=10000)

In [370]:
log1.score(X_train_p,y_train)

0.7882882882882883

In [371]:
log1.score(X_test_p,y_test)

0.852017937219731

In [372]:
param = {'penalty':['l2','l1'],
         'C': np.logspace(0,4,10)}
param1 = {'penalty':['l2'],
          'C': np.logspace(0,4,10),
          'solver':['saga','liblinear']}

In [373]:
rf_random = RandomizedSearchCV(estimator = log1, param_distributions = param1, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

In [374]:
rf_random.fit(X_train,y_train)

C:\Users\pande\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 20 is smaller than n_iter=100. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    7.7s finished


RandomizedSearchCV(cv=5, estimator=LogisticRegression(max_iter=10000),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                                        'penalty': ['l2'],
                                        'solver': ['saga', 'liblinear']},
                   random_state=42, verbose=2)

In [375]:
rf_random.best_params_

{'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}

In [376]:
logc = LogisticRegression(solver='liblinear',max_iter=10000)
logc.fit(X_train,y_train)

LogisticRegression(max_iter=10000, solver='liblinear')

In [377]:
logc.score(X_train,y_train)

0.7912912912912913

In [378]:
logc.score(X_test,y_test)

0.8475336322869955

In [379]:
df_test = pd.read_csv("C:/Users/pande/Desktop/kaggle/test.csv")
X_pred = df_test["PassengerId"]
df_test['Age'] = df_test[["Age","Pclass"]].apply(replace_age,axis=1)
df_test.drop(["Cabin"],axis=1,inplace=True)
sex = pd.get_dummies(df_test['Sex'],drop_first=True)
embark = pd.get_dummies(df_test['Embarked'],drop_first=True)
df_test.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
df_test = pd.concat([df_test,sex,embark],axis=1)
df_test.drop(["PassengerId"],axis=1,inplace=True)

In [380]:
ypred = xgbc.predict(df_test)

In [381]:
df1 = pd.DataFrame(X_pred,columns=["PassengerId"])
df2 = pd.DataFrame(ypred,columns=['Survived'])
data = pd.concat([df1,df2],axis=1)

In [382]:
data["Survived"].value_counts()

0    303
1    115
Name: Survived, dtype: int64

In [383]:
data.to_csv("submission_1234567_.csv")

In [1]:
#this prediction has a rank of 2174 at kaggle competition from 16820 total submitted predictions